In [2]:
from transformers import pipeline

# generator = pipeline(task="automatic-speech-recognition")

2023-08-22 15:34:37.126673: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-22 15:34:37.135180: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 15:34:37.202722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 15:34:37.203818: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 15:34:37.873849: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

/opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2023-08-22 15:34:39,185] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
# generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

In [14]:
generator = pipeline(model="openai/whisper-large")
# generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

In [16]:
def test(file):
    return generator(file, language='chinese'  )

In [10]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
# from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")
# model.config.forced_decoder_ids = None
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task="transcribe")
# load dummy dataset and read audio files
def test(sample):
    ds = load_dataset(sample)
    sample = ds[0]["audio"]
    input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 
    
    # generate token ids
    predicted_ids = model.generate(input_features)
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
    #['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.<|endoftext|>']
    
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    #[' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.']


In [4]:
import gradio as gr

In [17]:
with gr.Blocks() as demo:
    gr.Markdown("## 测试whisper模型")
    with gr.Row():
        inp = gr.Audio(source="microphone",label="麦克风",show_label=True,type='filepath')
        out = gr.Textbox()
    btn = gr.Button("Run")
    btn.click(fn=test, inputs=inp, outputs=out)

demo.launch(show_error=True)

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
